<img src="./assets/mellea_banner.png" width="80%"/>

# Mellea workshop - [ibm.biz/mellea-workshop](https://ibm.biz/mellea-workshop)

[Notebook on github](https://github.com/remiserra/mellea-workshop/blob/main/mellea_workshop.ipynb),
[Notebook on colab](https://colab.research.google.com/github/remiserra/mellea-workshop/blob/main/mellea_workshop.ipynb)

This workshop will introduce the basics of Generative Computing through a series of labs following a use-case.

During this workshop, we will:
1. Get up an running with Mellea.
2. See the Instruct - Validate - Repair pattern in action.
3. Encapsulate LLM calls using a functional interface via Mellea's `@generative` decorator.
4. Combine Mellea and Docling to write a generative program that operates over documents.
5. Leverage a vision model to analyse an image.

# Warmup

Run the first cell during our introduction. The first cell will:
 * download an install ollama on your Colab instance
 * pull the granite4 and granite-vision model weights from ollama
 * pull the docling model weights

This should take about 10 minutes.

In [ ]:
print("Install ollama")
# - When running locally, install ollama app from https://ollama.com, and the service should start automatically with mac/windows
# - When running on colab uncomment the next two lines to install and start ollama in the colab environment:
import subprocess
result = subprocess.run(["ollama", "--version"], capture_output=True, text=True)
if result.returncode == 0:
    print("Ollama already installed")
else:
    !curl -fsSL https://ollama.com/install.sh | sh > /dev/null
    !nohup ollama serve >/dev/null 2>&1 &

print("Download model weights")
# Download model weights for the Granite 4 and Granite vision models
!ollama pull ibm/granite4:micro
!ollama pull granite3.2-vision
print("Install Mellea with Docling")
!uv pip install mellea docling jupyter ipywidgets 
print("Run docling once to download model weights")
from mellea.stdlib.docs.richdocument import RichDocument
rd = RichDocument.from_document_file("https://arxiv.org/pdf/1906.04043")

# Some UI niceness
# from IPython.display import HTML, display  # noqa: E402

# def set_css():
#     display(HTML("\n<style>\n pre{\n white-space: pre-wrap;\n}\n</style>\n"))

# get_ipython().events.register("pre_run_cell", set_css)

print("Ready!")

# Introduction to Mellea.ai

## *“The second most remarkable thing about LLMs is that they can understand and produce fluent natural language. The most remarkable thing about them is that they do the wrong thing 5-50% of the time”*

### > **We need structures** that make it a first-class concern **to check our work**, and ideally, we should build new tools to allow interrogation of certainty.

<hr/>

## What is “generative computing”?

*Generative computing* is the idea that **LLMs** can function **as computing elements** that are a part of, not separate from, the rest of computer science.

Like computing in general, this will have many facets, spanning engineering practice, system design, theory, hardware, software, etc.

Many aspects are already emerging in the field, but they could use a nudge, and **we’re building tools to accelerate progress**.

<hr/>

## Mellea.ai

### Local-first Capability
Generative programs can do Big Model Things without Big Model hardware.

### Robust and Composable
Requirement-driven inference pipelines result in rock-solid libraries and deployable apps.

### Ready to Scale
Swap out inference engines and models with ease. Massively scale inference in one line of code.

### Open & Permissive
The open inference stack for building generative programs on open weight models.

<hr/>

# Workshop: Applied Principles for Mellea.ai

5 steps to showcase Mellea features with a unique workshop:

### 01. Easy prompting
Automated response via LLM

### 02. Validate & Retry
Generated response guided-verification

### 03. Function-Generation
Generative functions through desciptions

### 04. Embedded Document processing
Easy integration of document processing pipeline

### 05. Vision Model
Analyse images with vision models

<hr/>

## Use-case overview

Let's sell T-Shirts on our [Mellea.ai Store](https://mellea-ai-store.figma.site/)!

<iframe src="https://mellea-ai-store.figma.site/" width="100%" height=400></iframe>

Our objective it to facilitate returns

<hr/>

# Lab 1: Hello, Mellea!

## Mellea features

- Run locally with Ollama
- Get started with 3 lines of code

## Use-case illustration

We will prepare the welcome message for our apparel retail website.

## Lab guidelines

Running `mellea.start_session()` initialize a new `MelleaSession`. The session holds three things:
1. The model to use for this session. In this tutorial we will use granite3.3:8b.
2. An inference engine; i.e., the code that actually calls our model. We will be using ollama, but you can also use Huggingface or any OpenAI-compatible endpoint.
3. A `Context`, which tells Mellea how to remember context between requests. This is sometimes called the "Message History" in other frameworks. Throughout this tutorial, we will be using a `SimpleContext`. In `SimpleContext`s, **every request starts with a fresh context**. There is no preserved chat history between requests. Mellea provides other types of context, but today we will not be using those features. See the Tutorial for further details.

## Let's experiment!

In [ ]:
import mellea

m = mellea.start_session()

answer = m.chat("Please write a welcome message for an apparel website")

print(answer.content)

<hr/>

# Lab 2: Instruct-Validate-Retry

## Mellea features

Instruct-Validate-Repair is a design pattern for building robust automation using LLMs. The idea is simple:
1. Instruct the model to perform a task and specify requirements on the output of the task.
2. Validate that these requirements are satisfied by the model's output.
3. If any requirements fail, try to repair.

## Use-case illustration

We will prepare an e-mail answer to a return request, and check for specific aspects in the generated email.

## Lab guidelines

- Build a requirements list with the `req()` and `check()` constructors.
- use the `instruct()` method to ask Mellea to perform an instruction and retry until the requirements are met.


## Let's experiment!

In [ ]:
# We have the client name and request as inputs
client_name = "Joseph"
client_request = "Hi, I just received my Amanita T-sirt, it's really nice ! Unforunately it is too small for me, how can I return it ?"

In [ ]:
# We have instructions about the information to include in our emails
email_prompt = """
You are an automated customer service representative for an apparel boutique called Mellea Store. Your task is to generate a formal, polite, and email-ready response to customers who contact the shop regarding a product return. 

Follow these rules when writing your answer:
- Always reply using professional and courteous language suitable for an email. 
- Address the customer respectfully using “Hello” and close with “Regards” followed by “Mellea Store Team” 
- Include the standard French return policy terms compliant with consumer law. 
- The answer is concise. The lenght of the answer should be 120 words maximum.  

Your email must contain the following sections: 

1. Acknowledgement of the request
Confirm receipt of the return request in a friendly but formal tone.

2. Legal return period
State that customers have 14 days from receipt of goods to exercise their right of withdrawal (article L221-18 du Code de la consommation).

3. Product condition 
Specify that returned items must be new, unwashed, unworn, with tags and in original packaging.

4. Return procedure 
Explain clearly that the customer must:
- Include the original delivery note inside the parcel.
- Send it back to the address indicated on the label: 17 Avenue de l'Europe, 92270 Bois-Colombes.

5. Refund and return costs
Indicate that:
- Return shipping is free only if an error or defect occurred.
- Otherwise, it is at the customer's expense.
- Refunds are processed within 10-14 business days upon receipt and inspection of goods.

6. Final reassurance
End the message by offering support for further questions and thanking the customer for their trust.
The output must be a single, ready-to-send email formatted with correct paragraph spacing and without bullet points.
"""

In [ ]:
import mellea
from mellea.stdlib.requirement import req
from mellea.stdlib.sampling import RejectionSamplingStrategy

m = mellea.start_session()

# Add requirements to verify the generated email
requirements_list = [
    req("..."),
]

full_prompt = (
    email_prompt
    + """
    
    Client name:
    {{client_name}} 
    
    Client request:
    {{client_request}}
    """
)

email_candidate = m.instruct(
    description=full_prompt,
    requirements=requirements_list,
    strategy=RejectionSamplingStrategy(loop_budget=5),
    user_variables={"client_name": client_name, "client_request": client_request},
    return_sampling_results=True,
)

print(email_candidate.result)

## Solution

In [ ]:
import mellea
from mellea.stdlib.requirement import check, req, simple_validate
from mellea.stdlib.sampling import RejectionSamplingStrategy

m = mellea.start_session()

def generate_email(prompt, name, request):
    full_prompt = (
        prompt
        + """
        
        Client name:
        {{client_name}} 
        
        Client request:
        {{client_request}}
        """
    )

    requirements_list = [
        req("The email should have a salutation"),
        req(
            "Include the website url: mellea.ai",
            validation_fn=simple_validate(lambda x: "mellea.ai" in x),
        ),
        check("Make sure the mail is in correct English"),
    ]

    email_candidate = m.instruct(
        description=full_prompt,
        requirements=requirements_list,
        strategy=RejectionSamplingStrategy(loop_budget=5),
        user_variables={"client_name": name, "client_request": request},
        return_sampling_results=True,
    )
    if email_candidate.success:
        return email_candidate.result
    else:
        return email_candidate.sample_generations[0].value


print(generate_email(email_prompt, client_name, client_request))

<hr/>

# Lab 3: Generative Stubs

## Mellea features

In classical programming, pure (stateless) functions are a simple and powerful abstraction. A pure function takes inputs, computes outputs, and has no side effects. Generative programs can also use functions as abstraction boundaries, but in a generative program the meaning of the function can be given by an LLM instead of an interpreter or compiler. This is the idea behind a GenerativeSlot.

A GenerativeSlot is a function whose implementation is provided by an LLM. In Mellea, you define these using the `@generative` decorator. The function signature specifies the interface, and the docstring (or type annotations) guide the LLM in producing the output. Let's start with a simple example of a sentiment classifier using the Generative interface.

## Use-case illustration

Setup a classifier to 
- detect the sentiment of the client's email
- detect the reason for return 
and use this information to customize our answer

## Lab guidelines

- use the `@generative` annotation to create a runnable method from a text description
- determine possible outputs with the `Literal[]` type annnotation

## Let's experiment!

In [ ]:
import mellea
from mellea import generative
from typing import Literal

m = mellea.start_session()


@generative
def sentiment_classifier(text: str) -> None | Literal["positive", "negative"]:
    """Determine if the sentiment of `text` is positive or negative."""


sentiment = sentiment_classifier(m, text="The weather in Orlando is beautiful today!")
print(f"Detected sentiment: {sentiment}")

## Solution

In [ ]:
import mellea
from mellea import generative
from typing import Literal

m = mellea.start_session()


@generative
def sentiment_classifier(text: str) -> None | Literal["positive", "negative"]:
    """Determine if the sentiment of `text` is positive or negative."""


@generative
def return_reason_detection(text: str) -> None | Literal["size", "color"]:
    """Determine if the reason for the product return described in `text` is size or color."""


sentiment = sentiment_classifier(m, text=client_request)
print(f"Detected sentiment: {sentiment}")

reason = return_reason_detection(m, text=client_request)
print(f"Detected reason: {reason}")

advanced_prompt = email_prompt

if reason == "size":
    advanced_prompt = (
        advanced_prompt
        + """
7. Additional information
Mention the available sizes: XS, S, M, L, XL, XXL
    """
    )
elif reason == "color":
    advanced_prompt = (
        advanced_prompt
        + """
7. Additional information
Mention the available colors: Black, Blue, White, Yellow
    """
    )

print(advanced_prompt)

print(generate_email(advanced_prompt, client_name, client_request))

<hr/>

# Lab 4: Docling and Mellea

## Mellea features

In this lab, we will use both Docling and Mellea to extract data from a PDF.

## Use-case illustration

Use docling to read the invoice attached by the client and check for the product reference.

## Lab guidelines

- use the `RichDocument.from_document_file` to load a pdf file
- use `get_tables()` to retrieve the tables in the document
- use `m.query()` to ask a question on the table

## Let's experiment!

In [ ]:
from mellea.stdlib.docs.richdocument import RichDocument
from mellea.stdlib.docs.richdocument import Table

rd = RichDocument.from_document_file("./assets/Mellea Store - Invoice.pdf")

table1: Table = rd.get_tables()[0]
print(table1.to_markdown())

## Solution

In [ ]:
import mellea
from mellea.stdlib.docs.richdocument import RichDocument
from mellea.stdlib.docs.richdocument import Table

rd = RichDocument.from_document_file("./assets/Mellea Store - Invoice.pdf")

table1: Table = rd.get_tables()[0]
print(table1.to_markdown())

m = mellea.start_session()
ref = m.query(table1, "what is the product number ?")
print(ref)

print(f'Product number: {ref.value.split("#")[1]}')

<hr/>

# Lab 5: Vision models

## Mellea features

Analyse an image by leveraging a vision model directly in Mellea <img src="./assets/mellea_pointing_up.jpg" height=30 />

## Use-case illustration

If the client has lost the invoce, we can use a vision model to read a photo provided by the client and validate the item to be returned.

We can use the following images:

[photo-mellea-shoe](./assets/photo-mellea-shoe.png)
<img src="./assets/photo-mellea-shoe.png" width=200 />
[photo-mellea-tshirt](./assets/photo-mellea-tshirt.png)
<img src="./assets/photo-mellea-tshirt.png" width=200 />

## Lab guidelines

- start a mellea session with a model vision
- load an image
- pass the image as context to the `instruct()` method

## Let's experiment!

In [ ]:
import mellea
from PIL import Image

# Initiate a session with a model vision
m = mellea.start_session(model_id="granite3.2-vision")

# load image
test_img = Image.open("./assets/mellea_pointing_up.jpg")

# ask a question about the image
res = m.instruct("Is the subject in the image smiling?", images=[test_img])
print(f"Result:{res}")

## Solution

In [ ]:
import mellea
from PIL import Image

# Initiate a session with a model vision
m = mellea.start_session(model_id="granite3.2-vision")

# load image
test_img = Image.open("./assets/photo-mellea-shoe.png")

# ask a question about the 1st image
res = m.instruct("Is this a T-shirt ?", images=[test_img])
print(f"Photo 1 result: {res.value}")

# ask a question about the 2nd image
print(
    f"Photo 2 result: {m.instruct("Is this a T-shirt ?", images=[Image.open("./assets/photo-mellea-tshirt.png")])}"
)